In [1]:
!pip install yfinance pandas google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peewee: filename=peewee-3.17.9-cp310-cp310-linux_x86_64.whl size=299100 sha256=de5e517a8a0331a1ee332f706f18cdf566af7979c9a9da8a265c6fed7519866a
  Stored in directory: /home/jupyter/.cache/pip/wheels/fd/fd/5e/90b9ec95da4fd6c96237b580ce74f89d6bdea547ad151ab5f4
Successfully built peewee


In [ ]:
# If not already installed in Vertex AI notebook
# !pip install yfinance google-cloud-storage

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from google.cloud import storage
import os

# --- Get last Monday (e.g., March 31st if current week is April 1–7) --- #
today = datetime.now()
last_monday = today - timedelta(days=today.weekday() + 7)  # Go back to *last* Monday

# --- Configuration --- #
TICKERS = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "TSLA": "Tesla, Inc.",
    "GOOGL": "Alphabet Inc.",
    "AMZN": "Amazon.com, Inc.",
    "NVDA": "NVIDIA Corporation",
    "META": "Meta Platforms, Inc.",
    "JPM": "JPMorgan Chase & Co.",
    "NFLX": "Netflix, Inc.",
    "INTC": "Intel Corporation"
}

START_DATE = (last_monday - timedelta(days=8*365)).strftime('%Y-%m-%d')
END_DATE = last_monday.strftime('%Y-%m-%d')
LOCAL_FILE = f"stocks.json"
LOCAL_PATH = f"/tmp/{LOCAL_FILE}"
GCS_BUCKET_NAME = "stock-bucket5"
GCS_DESTINATION = f"stock_data/date={END_DATE}/{LOCAL_FILE}"


# --- Fetch and combine --- #
all_data = []

for ticker, name in TICKERS.items():
    print(f"Fetching {ticker} from {START_DATE} to {END_DATE}...")
    stock = yf.Ticker(ticker)
    df = stock.history(start=START_DATE, end=END_DATE)

    if df.empty:
        print(f"No data for {ticker}")
        continue

    df = df.reset_index()
    df['Ticker'] = ticker
    df['Company'] = name
    all_data.append(df)

# --- Combine and save locally --- #
if all_data:
    result = pd.concat(all_data, ignore_index=True)
    result['Date'] = result['Date'].astype(str)
    result.to_json(LOCAL_PATH, orient='records', lines=True)
    print(f"✅ JSON saved locally at: {LOCAL_PATH}")

    # --- Upload to GCS --- #
    def upload_to_gcs(local_file, bucket_name, gcs_path):
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(gcs_path)
        blob.upload_from_filename(local_file)
        print(f"✅ Uploaded to GCS: gs://{bucket_name}/{gcs_path}")

    upload_to_gcs(LOCAL_PATH, GCS_BUCKET_NAME, GCS_DESTINATION)

else:
    print("🚫 No stock data was fetched.")
